# Generate Color Bins For Each Layer

### 0. Import libraries and create empty dictionaries

In [21]:
import geopandas as gpd
import numpy as np
import mapclassify as mc
import matplotlib.pyplot as plt

In [22]:
total_dict = {}
county_dict = {}
state_dict = {}
dph_illinois_dict = {}
who_world_dict = {}
vul_dict = {}

In [23]:
dynamic_dict = {}
dynamic_county_dict = {}
dynamic_state_dict = {}
dynamic_illinois_dict = {}

### 1. For US County-level Data

#### 1.0 Load Data

In [24]:
counties_gdf = gpd.read_file('nyt_counties_data.geojson')

In [25]:
counties_gdf.head(5)

,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,start,end,dt_unit,change_ts,geometry
0,Bladen,North Carolina,33778,37017,2020-04-03,2020-05-06,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",5499.0,0.0,95.0,0.0,2020-01-21,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ..."
1,Stanly,North Carolina,61114,37167,2020-03-20,2020-04-09,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",11752.0,0.0,191.0,0.0,2020-01-21,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ..."
2,Summit,Ohio,541810,39153,2020-03-14,2020-03-27,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",64072.0,237.0,1144.0,0.0,2020-01-21,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ..."
3,Sullivan,Pennsylvania,6177,42113,2020-04-04,2020-10-14,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",600.0,1.0,24.0,0.0,2020-01-21,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ..."
4,Upshur,Texas,40769,48459,2020-03-20,2020-07-25,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",5856.0,0.0,144.0,0.0,2020-01-21,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ..."


In [26]:
case = counties_gdf['today_case']
death = counties_gdf['today_death']
case_per_100k_capita = counties_gdf['today_case']/(counties_gdf['population']/100000)
death_per_100k_capita = counties_gdf['today_death']/(counties_gdf['population']/100000)
# death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
# death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [27]:
log_case = case.apply(lambda x: np.log(x + 1))
log_death = death.apply(lambda x: np.log(x + 1))
log_case_per_100k_capita = case_per_100k_capita.apply(lambda x: np.log(x + 1))
log_death_per_100k_capita = death.apply(lambda x: np.log(x + 1))

#### 1.1 Get Constant Classes from today's data

In [28]:
def create_dict(column):
    tmp_dict = {}
    tmp_dict['Quantiles'] = {
        'bins': ','.join(mc.Quantiles(column, k=6).bins.astype(str)),
        'k': 6
    }
    tmp_dict['FisherJenks'] = {
        'bins': ','.join(mc.FisherJenks(column, k=6).bins.astype(str)),
        'k': 6
    }
    tmp_dict['NaturalBreaks'] = {
        'bins': ','.join(mc.NaturalBreaks(column, k=6).bins.astype(str)),
        'k': 6
    }
    return tmp_dict

In [29]:
county_dict['case'] = {}
county_dict['death'] = {}
county_dict['case_per_100k_capita'] = {}
county_dict['death_per_100k_capita'] = {}

In [30]:
county_dict['case']['nolog'] = create_dict(case)
county_dict['case']['log'] = create_dict(log_case)
county_dict['death']['nolog'] = create_dict(death)
county_dict['death']['log'] = create_dict(log_death)
county_dict['case_per_100k_capita']['nolog'] = create_dict(case_per_100k_capita)
county_dict['case_per_100k_capita']['log'] = create_dict(log_case_per_100k_capita)
county_dict['death_per_100k_capita']['nolog'] = create_dict(death_per_100k_capita)
county_dict['death_per_100k_capita']['log'] = create_dict(log_death_per_100k_capita)


In [31]:
county_dict

{'case': {'nolog': {'Quantiles': {'bins': '1005.0000000000002,2166.0000000000005,3784.0,6817.000000000001,16128.000000000013,1503380.0',
    'k': 6},
   'FisherJenks': {'bins': '26748.0,88522.0,228752.0,406991.0,752986.0,1503380.0',
    'k': 6},
   'NaturalBreaks': {'bins': '26748.0,88522.0,228752.0,406991.0,752986.0,1503380.0',
    'k': 6}},
  'log': {'Quantiles': {'bins': '6.913737350659685,7.68109900153636,8.23880116587155,8.827321452697849,9.688374172917184,14.223227129625158',
    'k': 6},
   'FisherJenks': {'bins': '2.1972245773362196,6.498282149476434,7.851661177889265,9.023528749909364,10.424065467744796,14.223227129625158',
    'k': 6},
   'NaturalBreaks': {'bins': '2.1972245773362196,6.498282149476434,7.865955413933502,9.056839480990421,10.478554778363584,14.223227129625158',
    'k': 6}}},
 'death': {'nolog': {'Quantiles': {'bins': '14.0,37.0,65.0,111.0,256.0,34621.0',
    'k': 6},
   'FisherJenks': {'bins': '416.0,1399.0,3466.0,6014.0,12352.0,34621.0',
    'k': 6},
   'Natu

### 2. For US State-level Data

#### 2.0 Load Data

In [32]:
state_gdf = gpd.read_file('nyt_states_data.geojson')

In [33]:
state_gdf.head(5)

,index,NAME,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,start,end,dt_unit,change_ts,geometry
0,0,West Virginia,1829054,54,2020-03-17,2020-03-29,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",278171,1038,4518,0,2020-01-21,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","POLYGON ((-80.51899 40.63880, -80.51934 39.721..."
1,1,Florida,20598139,12,2020-03-01,2020-03-06,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",3659255,0,60334,0,2020-01-21,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","MULTIPOLYGON (((-82.98748 24.62538, -82.89993 ..."
2,2,Illinois,12821497,17,2020-01-24,2020-03-17,"0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",1718384,0,28832,0,2020-01-21,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,-1.0,-1....","POLYGON ((-90.64284 42.50848, -87.01994 42.493..."
3,3,Minnesota,5527358,27,2020-03-06,2020-03-21,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",815132,0,8952,0,2020-01-21,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","POLYGON ((-97.22872 49.00056, -95.15371 48.998..."
4,4,Maryland,6003435,24,2020-03-05,2020-03-18,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",566313,727,10974,9,2020-01-21,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","POLYGON ((-77.71952 39.32131, -77.76086 39.340..."


#### 2.1 Get Constant Classes from today's data

In [34]:
state_case = state_gdf['today_case']
state_death = state_gdf['today_death']
state_case_per_100k_capita = state_gdf['today_case']/(state_gdf['population']/100000)
state_death_per_100k_capita = state_gdf['today_death']/(state_gdf['population']/100000)
#death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
#death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [35]:
state_death_per_100k_capita = state_death_per_100k_capita.replace(np.nan,0)
state_death_per_100k_capita = state_death_per_100k_capita.replace(np.inf,0)
state_case_per_100k_capita = state_case_per_100k_capita.replace(np.nan,0)
state_case_per_100k_capita = state_case_per_100k_capita.replace(np.inf,0)

In [36]:
state_log_case = state_case.apply(lambda x: np.log(x + 1))
state_log_death = state_death.apply(lambda x: np.log(x + 1))
state_log_case_per_100k_capita = state_case_per_100k_capita.apply(lambda x: np.log(x + 1))
state_log_death_per_100k_capita =state_death_per_100k_capita.apply(lambda x: np.log(x + 1))

In [37]:
state_dict['case'] = {}
state_dict['death'] = {}
state_dict['case_per_100k_capita'] = {}
state_dict['death_per_100k_capita'] = {}

In [38]:
state_dict['case']['nolog'] = create_dict(state_case)
state_dict['case']['log'] = create_dict(state_log_case)
state_dict['death']['nolog'] = create_dict(state_death)
state_dict['death']['log'] = create_dict(state_log_death)
state_dict['case_per_100k_capita']['nolog'] = create_dict(state_case_per_100k_capita)
state_dict['case_per_100k_capita']['log'] = create_dict(state_log_case_per_100k_capita)
state_dict['death_per_100k_capita']['nolog'] = create_dict(state_death_per_100k_capita)
state_dict['death_per_100k_capita']['log'] = create_dict(state_log_death_per_100k_capita)

In [39]:
state_dict

{'case': {'nolog': {'Quantiles': {'bins': '141182.0,289282.0,560717.0,861480.0000000001,1307187.0000000014,4961782.0',
    'k': 6},
   'FisherJenks': {'bins': '455163,1033866,1718384,2587606,4252903,4961782',
    'k': 6},
   'NaturalBreaks': {'bins': '455163.0,1033866.0,1718384.0,2587606.0,4252903.0,4961782.0',
    'k': 6}},
  'log': {'Quantiles': {'bins': '11.857812200192459,12.575160726707395,13.236973384325214,13.66640828015091,14.083388823119181,15.417275709911614',
    'k': 6},
   'FisherJenks': {'bins': '5.840641657373398,10.652873279543263,12.297840193983275,13.381969996907825,14.356895454208304,15.417275709911614',
    'k': 6},
   'NaturalBreaks': {'bins': '5.840641657373398,10.652873279543263,12.297840193983275,13.381969996907825,14.356895454208304,15.417275709911614',
    'k': 6}}},
 'death': {'nolog': {'Quantiles': {'bins': '1597.0000000000005,3647.000000000003,8790.0,13841.000000000002,23951.000000000007,72698.0',
    'k': 6},
   'FisherJenks': {'bins': '5115,10974,21409,31

### 7. Summary and ouput

In [40]:
import json
total_dict['county'] = county_dict
total_dict['state'] = state_dict
with open('classes_nyt.json','w') as json_file:
    json.dump(total_dict, json_file)
print('done')

done
